In [96]:
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import os
from tabulate import tabulate
import pandas as pd
import warnings

warnings.filterwarnings("ignore")


def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def LIMA(alpha, Non, Noff):
    sig = np.sqrt(2) * np.sqrt(
        Non * np.log((1 + alpha) / alpha * (Non / (Non + Noff)))
        + Noff * np.log((1 + alpha) * Noff / (Non + Noff))
    )
    if type(sig) is np.ndarray:
        sig[np.where((Non - Noff * alpha) < 0)] = -sig[
            np.where((Non - Noff * alpha) < 0)
        ]
    else:
        if (Non - Noff * alpha) < 0:
            sig = -sig
    return sig


def twoPointAngle(theta1, theta2, phi1, phi2):
    acosangle = np.sin(np.deg2rad(theta1)) * np.sin(np.deg2rad(theta2)) * np.cos(
        np.deg2rad(phi1 - phi2)
    ) + np.cos(np.deg2rad(theta1)) * np.cos(np.deg2rad(theta2))
    acosangle[acosangle > 1] = 1
    acosangle[acosangle <-1] = -1
    return np.rad2deg(np.arccos(acosangle))



In [2]:
Exptdata = np.load(
    "/home2/hky/github/Gamma_Energy/Exptdata/CrabCut_23_05_07_summdcut/cutedData_E_isgamma_RaDec.npz"
)

gamma_cut = 0.005
need = np.where(Exptdata["isgamma_5"] > gamma_cut)
Exptdata = {key: Exptdata[key][need] for key in Exptdata}

In [3]:
NSIDE = 2**9
NPIX = hp.nside2npix(NSIDE)
reso = 3
drawdeg = 3
binsnumber = int(drawdeg * 60 / reso * 2)
TeVcat = np.loadtxt("/home2/hky/github/Gamma_Energy/Crab/TeVcat.log")
Ra_TeVcat = (
    TeVcat[:, 0] / 24 + TeVcat[:, 1] / 24 / 60 + TeVcat[:, 2] / 24 / 60 / 60
) * 360
Dec_TeVcat = TeVcat[:, 3] + TeVcat[:, 4] / 60 + TeVcat[:, 5] / 60 / 60
tablelist = list()

In [4]:
Energymin=10

In [5]:
print(Energymin)
On = np.zeros(NPIX)
Off = np.zeros(NPIX)
need = np.where((Exptdata["energy"] > Energymin))
np.add.at(
    On,
    hp.ang2pix(NSIDE, Exptdata["Ra"][need], Exptdata["Dec"][need], lonlat=True),
    1,
)
for i in range(20):
    np.add.at(
        Off,
        hp.ang2pix(
            NSIDE,
            Exptdata[f"RaOff_{i}"][need],
            Exptdata[f"DecOff_{i}"][need],
            lonlat=True,
        ),
        1,
    )

10


In [6]:
smoothed=0.3

In [72]:
fwhm = np.deg2rad(smoothed)
On_smoothed = hp.smoothing(On, fwhm=fwhm)
Off_smoothed = hp.smoothing(Off, fwhm=fwhm)

In [122]:
sigma_hp = LIMA(0.05, On_smoothed, Off_smoothed)
sigma_hp[np.isnan(sigma_hp)] = 0
sigma_hp /= np.std(sigma_hp[On != 0])
pixneed = np.where(sigma_hp > 3.5)[0]

In [123]:
X = np.rad2deg(hp.pix2ang(NSIDE, pixneed)).T

In [124]:
from sklearn.cluster import DBSCAN

In [125]:
def similarity(x):
    distance = np.zeros([x.shape[0],x.shape[0]])
    for i in range(x.shape[0]):
        # print(i)
        distance[i] = twoPointAngle(
                    x[i,0], x[:,0], x[i,1], x[:,1]
                )
    return distance

In [126]:
distance=similarity(X)

In [131]:
model = DBSCAN(eps=3,min_samples=5,metric="precomputed")
yhat = model.fit_predict(distance)

In [132]:
np.unique(yhat)

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 17

In [137]:
sigma_hp[pixneed[np.where(yhat==0)]]

array([4.13367891, 3.90441515, 3.56110189, 3.70579016, 3.86320233,
       3.55885597, 3.88621564, 4.17254763, 3.54037884, 3.60069247,
       3.94479681, 3.5818261 , 4.0603468 , 3.83206554, 3.75341327,
       3.73288981, 4.24394835, 3.71417168, 3.66190237, 4.21053257,
       4.12337683, 3.65721083, 3.60758186, 3.62519379, 4.01482336,
       3.67977289, 4.0431432 , 3.73523446, 3.76196347, 3.56629729,
       3.53655976, 3.67699919, 4.15726249, 4.02458339, 4.06221694,
       3.56840463, 4.03632274, 3.5884454 , 3.50558886, 3.7823647 ,
       4.05032262, 3.53650535, 3.68700387, 3.51717515, 3.59654025,
       3.50562427, 3.9515422 , 3.5505055 , 3.77663384, 3.59960808,
       3.62580538, 3.96417993, 3.63544587, 3.67716978, 3.51511919,
       4.04950165, 3.90448805, 3.5167736 , 3.68214155, 3.65261528,
       3.53258265, 4.17750138, 3.51532628, 3.84723744, 4.42411792,
       3.76996505, 4.40238966, 4.10297457, 3.56055842, 3.61384928,
       3.66316106, 3.64110755, 3.81977723, 3.57049449, 4.04742

In [135]:
pixneed[yhat==0][np.argmax(sigma_hp[pixneed][yhat==0])]

38832

In [136]:
sigma_hp[38832]

5.51937356979823